In [1]:
import requests
import json

In [2]:
api_key = "HZVHfiBGppozkKIAeLZJuZzik"
BBOX = "[[[-25.9973998712, 25.3439083708], [44.6049090453, 71.2982931893]]]"

In [3]:
def parse_bbox(bbox_str):
    """Safely parse [[[min_long, max_long],[min_lat, max_lat]]]"""
    try:
        data = json.loads(bbox_str)
        # data[0][0] is [min_long, max_long], data[0][1] is [min_lat, max_lat]
        return data[0][0][0], data[0][0][1], data[0][1][0], data[0][1][1]
    except Exception as e:
        raise ValueError(f"Failed to parse BBOX: {e}")

In [4]:
min_long, max_long, min_lat, max_lat = parse_bbox(BBOX)
max_lat, min_lat, max_long, min_long

(71.2982931893, 44.6049090453, 25.3439083708, -25.9973998712)

In [35]:
import openmeteo_requests

import pandas as pd
import requests_cache
from retry_requests import retry

# Setup the Open-Meteo API client with cache and retry on error
cache_session = requests_cache.CachedSession('.cache', expire_after = 3600)
retry_session = retry(cache_session, retries = 5, backoff_factor = 0.2)
openmeteo = openmeteo_requests.Client(session = retry_session)

# Make sure all required weather variables are listed here
# The order of variables in hourly or daily is important to assign them correctly below
url = "https://marine-api.open-meteo.com/v1/marine"
params = {
	"latitude": 53.95,
	"longitude": 10.866667,
	"hourly": ["wave_height", "wave_direction", "tertiary_swell_wave_direction", "wave_period", "wave_peak_period", "sea_level_height_msl", "sea_surface_temperature", "ocean_current_direction", "ocean_current_velocity", "swell_wave_peak_period", "swell_wave_period", "swell_wave_direction", "swell_wave_height", "wind_wave_peak_period", "wind_wave_period", "wind_wave_direction", "wind_wave_height", "tertiary_swell_wave_period", "tertiary_swell_wave_height", "secondary_swell_wave_direction", "secondary_swell_wave_period", "secondary_swell_wave_height"],
	# "current": ["wave_height", "wave_direction", "wave_period", "wave_peak_period", "wind_wave_height", "wind_wave_direction", "wind_wave_period", "wind_wave_peak_period", "swell_wave_height", "swell_wave_direction", "swell_wave_period", "swell_wave_peak_period", "secondary_swell_wave_height", "secondary_swell_wave_period", "secondary_swell_wave_direction", "tertiary_swell_wave_height", "tertiary_swell_wave_period", "tertiary_swell_wave_direction", "sea_level_height_msl", "sea_surface_temperature", "ocean_current_velocity", "ocean_current_direction"],
	"forecast_days": 3,
	"wind_speed_unit": "ms",
}
responses = openmeteo.weather_api(url, params=params)

# Process first location. Add a for-loop for multiple locations or weather models
response = responses[0]
print(f"Coordinates: {response.Latitude()}°N {response.Longitude()}°E")
print(f"Elevation: {response.Elevation()} m asl")
print(f"Timezone difference to GMT+0: {response.UtcOffsetSeconds()}s")


Coordinates: 53.958335876464844°N 10.875015258789062°E
Elevation: 5.0 m asl
Timezone difference to GMT+0: 0s


In [36]:
# Process current data. The order of variables needs to be the same as requested.
current = response.Current()
current_wave_height = current.Variables(0).Value()
current_wave_direction = current.Variables(1).Value()
current_wave_period = current.Variables(2).Value()
current_wave_peak_period = current.Variables(3).Value()
current_wind_wave_height = current.Variables(4).Value()
current_wind_wave_direction = current.Variables(5).Value()
current_wind_wave_period = current.Variables(6).Value()
current_wind_wave_peak_period = current.Variables(7).Value()
current_swell_wave_height = current.Variables(8).Value()
current_swell_wave_direction = current.Variables(9).Value()
current_swell_wave_period = current.Variables(10).Value()
current_swell_wave_peak_period = current.Variables(11).Value()
current_secondary_swell_wave_height = current.Variables(12).Value()
current_secondary_swell_wave_period = current.Variables(13).Value()
current_secondary_swell_wave_direction = current.Variables(14).Value()
current_tertiary_swell_wave_height = current.Variables(15).Value()
current_tertiary_swell_wave_period = current.Variables(16).Value()
current_tertiary_swell_wave_direction = current.Variables(17).Value()
current_sea_level_height_msl = current.Variables(18).Value()
current_sea_surface_temperature = current.Variables(19).Value()
current_ocean_current_velocity = current.Variables(20).Value()
current_ocean_current_direction = current.Variables(21).Value()

print(f"\nCurrent time: {current.Time()}")
print(f"Current wave_height: {current_wave_height}")
print(f"Current wave_direction: {current_wave_direction}")
print(f"Current wave_period: {current_wave_period}")
print(f"Current wave_peak_period: {current_wave_peak_period}")
print(f"Current wind_wave_height: {current_wind_wave_height}")
print(f"Current wind_wave_direction: {current_wind_wave_direction}")
print(f"Current wind_wave_period: {current_wind_wave_period}")
print(f"Current wind_wave_peak_period: {current_wind_wave_peak_period}")
print(f"Current swell_wave_height: {current_swell_wave_height}")
print(f"Current swell_wave_direction: {current_swell_wave_direction}")
print(f"Current swell_wave_period: {current_swell_wave_period}")
print(f"Current swell_wave_peak_period: {current_swell_wave_peak_period}")
print(f"Current secondary_swell_wave_height: {current_secondary_swell_wave_height}")
print(f"Current secondary_swell_wave_period: {current_secondary_swell_wave_period}")
print(f"Current secondary_swell_wave_direction: {current_secondary_swell_wave_direction}")
print(f"Current tertiary_swell_wave_height: {current_tertiary_swell_wave_height}")
print(f"Current tertiary_swell_wave_period: {current_tertiary_swell_wave_period}")
print(f"Current tertiary_swell_wave_direction: {current_tertiary_swell_wave_direction}")
print(f"Current sea_level_height_msl: {current_sea_level_height_msl}")
print(f"Current sea_surface_temperature: {current_sea_surface_temperature}")
print(f"Current ocean_current_velocity: {current_ocean_current_velocity}")
print(f"Current ocean_current_direction: {current_ocean_current_direction}")

AttributeError: 'NoneType' object has no attribute 'Variables'

In [32]:
str(current_wave_peak_period).encode("utf-8")

b'nan'

In [ ]:
# Process hourly data. The order of variables needs to be the same as requested.
hourly = response.Hourly()
hourly_wave_height = hourly.Variables(0).ValuesAsNumpy()
hourly_wave_direction = hourly.Variables(1).ValuesAsNumpy()
hourly_tertiary_swell_wave_direction = hourly.Variables(2).ValuesAsNumpy()
hourly_wave_period = hourly.Variables(3).ValuesAsNumpy()
hourly_wave_peak_period = hourly.Variables(4).ValuesAsNumpy()
hourly_sea_level_height_msl = hourly.Variables(5).ValuesAsNumpy()
hourly_sea_surface_temperature = hourly.Variables(6).ValuesAsNumpy()
hourly_ocean_current_direction = hourly.Variables(7).ValuesAsNumpy()
hourly_ocean_current_velocity = hourly.Variables(8).ValuesAsNumpy()
hourly_swell_wave_peak_period = hourly.Variables(9).ValuesAsNumpy()
hourly_swell_wave_period = hourly.Variables(10).ValuesAsNumpy()
hourly_swell_wave_direction = hourly.Variables(11).ValuesAsNumpy()
hourly_swell_wave_height = hourly.Variables(12).ValuesAsNumpy()
hourly_wind_wave_peak_period = hourly.Variables(13).ValuesAsNumpy()
hourly_wind_wave_period = hourly.Variables(14).ValuesAsNumpy()
hourly_wind_wave_direction = hourly.Variables(15).ValuesAsNumpy()
hourly_wind_wave_height = hourly.Variables(16).ValuesAsNumpy()
hourly_tertiary_swell_wave_period = hourly.Variables(17).ValuesAsNumpy()
hourly_tertiary_swell_wave_height = hourly.Variables(18).ValuesAsNumpy()
hourly_secondary_swell_wave_direction = hourly.Variables(19).ValuesAsNumpy()
hourly_secondary_swell_wave_period = hourly.Variables(20).ValuesAsNumpy()
hourly_secondary_swell_wave_height = hourly.Variables(21).ValuesAsNumpy()

hourly_data = {"date": pd.date_range(
	start = pd.to_datetime(hourly.Time(), unit = "s", utc = True),
	end =  pd.to_datetime(hourly.TimeEnd(), unit = "s", utc = True),
	freq = pd.Timedelta(seconds = hourly.Interval()),
	inclusive = "left"
)}

hourly_data["wave_height"] = hourly_wave_height
hourly_data["wave_direction"] = hourly_wave_direction
hourly_data["tertiary_swell_wave_direction"] = hourly_tertiary_swell_wave_direction
hourly_data["wave_period"] = hourly_wave_period
hourly_data["wave_peak_period"] = hourly_wave_peak_period
hourly_data["sea_level_height_msl"] = hourly_sea_level_height_msl
hourly_data["sea_surface_temperature"] = hourly_sea_surface_temperature
hourly_data["ocean_current_direction"] = hourly_ocean_current_direction
hourly_data["ocean_current_velocity"] = hourly_ocean_current_velocity
hourly_data["swell_wave_peak_period"] = hourly_swell_wave_peak_period
hourly_data["swell_wave_period"] = hourly_swell_wave_period
hourly_data["swell_wave_direction"] = hourly_swell_wave_direction
hourly_data["swell_wave_height"] = hourly_swell_wave_height
hourly_data["wind_wave_peak_period"] = hourly_wind_wave_peak_period
hourly_data["wind_wave_period"] = hourly_wind_wave_period
hourly_data["wind_wave_direction"] = hourly_wind_wave_direction
hourly_data["wind_wave_height"] = hourly_wind_wave_height
hourly_data["tertiary_swell_wave_period"] = hourly_tertiary_swell_wave_period
hourly_data["tertiary_swell_wave_height"] = hourly_tertiary_swell_wave_height
hourly_data["secondary_swell_wave_direction"] = hourly_secondary_swell_wave_direction
hourly_data["secondary_swell_wave_period"] = hourly_secondary_swell_wave_period
hourly_data["secondary_swell_wave_height"] = hourly_secondary_swell_wave_height

hourly_dataframe = pd.DataFrame(data = hourly_data)
print("\nHourly data\n", hourly_dataframe)


Hourly data
                         date  wave_height  wave_direction  \
0  2026-01-09 00:00:00+00:00         0.28           105.0   
1  2026-01-09 01:00:00+00:00         0.32            97.0   
2  2026-01-09 02:00:00+00:00         0.36            93.0   
3  2026-01-09 03:00:00+00:00         0.40            90.0   
4  2026-01-09 04:00:00+00:00         0.44            85.0   
..                       ...          ...             ...   
67 2026-01-11 19:00:00+00:00         0.08            43.0   
68 2026-01-11 20:00:00+00:00         0.08            44.0   
69 2026-01-11 21:00:00+00:00         0.08            45.0   
70 2026-01-11 22:00:00+00:00         0.06            46.0   
71 2026-01-11 23:00:00+00:00         0.06            47.0   

    tertiary_swell_wave_direction  wave_period  wave_peak_period  \
0                             NaN         2.40               NaN   
1                             NaN         2.60               NaN   
2                             NaN         2.80   

In [38]:
hourly.Interval()

3600

1768176000